# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [2]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 


from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import numpy as np
import pandas as pd


spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)
user_log.createOrReplaceTempView("user_log_table")
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id ""(empty string) NOT visit?

In [23]:
# TODO: write your code to answer question 1

pages_visited = spark.sql('''SELECT page FROM user_log_table 
                            WHERE userId == ""''').dropDuplicates()

pages_all = spark.sql('''SELECT page FROM user_log_table''').dropDuplicates()

pages_unseen = set(pages_all.collect()) - set(pages_visited.collect())

print("Unseen Pages:")
print(pages_unseen)

Unseen Pages:
{Row(page='Settings'), Row(page='Submit Upgrade'), Row(page='Error'), Row(page='Upgrade'), Row(page='Logout'), Row(page='Submit Downgrade'), Row(page='NextSong'), Row(page='Save Settings'), Row(page='Downgrade')}


# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

In [18]:
# SQL:
# - Familiarity for certain users
# - Perhaps code can be easier translated to Spark in Java or other scenarios


# DFs:
# - Familiarity for certain users
# - Perhaps code can be easier translated to pandas

# Question 3

How many female users do we have in the data set?

In [36]:
# TODO: write your code to answer question 3


user_log_distinct = spark.sql('''
                            SELECT userId, gender 
                            FROM user_log_table
                            ''').dropDuplicates()


user_log_distinct.createOrReplaceTempView("user_log_distinct")

user_gender_dist = spark.sql('''
                            SELECT gender, COUNT(userId)
                            FROM user_log_distinct
                            GROUP BY gender
                            ''')

user_gender_dist.show()

+------+-------------+
|gender|count(userId)|
+------+-------------+
|     F|          462|
|  null|            1|
|     M|          501|
+------+-------------+



# Question 4

How many songs were played from the most played artist?

In [39]:
# TODO: write your code to answer question 4

artist_popularity = spark.sql('''
                            SELECT artist, count(userId) AS playsCount
                            FROM user_log_table
                            WHERE page == "NextSong"
                            GROUP BY artist
                            ORDER BY playsCount DESC
                            ''')

artist_popularity.show()

+--------------------+----------+
|              artist|playsCount|
+--------------------+----------+
|            Coldplay|        83|
|       Kings Of Leon|        69|
|Florence + The Ma...|        52|
|            BjÃÂ¶rk|        46|
|       Dwight Yoakam|        45|
|       Justin Bieber|        43|
|      The Black Keys|        40|
|         OneRepublic|        37|
|                Muse|        36|
|        Jack Johnson|        36|
|           Radiohead|        31|
|        Taylor Swift|        29|
|Barry Tuckwell/Ac...|        28|
|          Lily Allen|        28|
|               Train|        28|
|           Daft Punk|        27|
|           Metallica|        27|
|          Nickelback|        27|
|          Kanye West|        26|
|Red Hot Chili Pep...|        24|
+--------------------+----------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [42]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = spark.sql("SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home FROM user_log_table \
            WHERE (page = 'NextSong') or (page = 'Home') \
            ")

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = spark.sql("SELECT *, SUM(is_home) OVER \
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_home_table")

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
spark.sql("SELECT AVG(count_results) FROM \
          (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

